In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat
from matplotlib import cm

In [ ]:
pStr = lambda p : "{:04.0f}".format(p)

# File Paths (Parareal Solution)
PpDir = lambda Ng, k , pTot, p : f"vp/parareal-ng-{Ng}-k-{k}-P{pStr(pTot)}/Proc_{pStr(p)}"
ParSolFile = lambda Ng, k, pTot, p : f"{PpDir(Ng, k, pTot, p)}/yend.npy" 
ParTimingFile = lambda Ng, k, pTot, p : f"{PpDir(Ng, k, pTot, p)}/runtime.json" 

# File Paths (Serial RK - Fine)
ERKFinepDir = f"vp/serial-fine-P{pStr(1)}/Proc_{pStr(0)}"
ERKFineSolFile = f"{ERKFinepDir}/yend.npy"
ERKFineTimingFile = f"{ERKFinepDir}/runtime.json"

# File Paths (Serial RK -- Reference)
ERKRefpDir = f"vp/serial-ref-P{pStr(1)}/Proc_{pStr(0)}"
ERKRefSolFile = f"{ERKRefpDir}/yend.npy"
ERKRefTimingFile = f"{ERKRefpDir}/runtime.json"

def extractTotalTime(timing_filepath):
    f = open(timing_filepath)
    data = json.load(f)
    f.close()
    return data["total"]

In [ ]:
# Experiment Parameters
Ngs = np.arange(1,4)
ks = np.arange(0,12)
pTot = 1
pFin = pTot - 1

sol2Realspace = lambda sol : np.real(np.fft.ifft(sol, n=None, axis=0)) # transform in x and take real part
solNorm = lambda sol : np.linalg.norm(sol2Realspace(sol).flatten()) # flatten and inf norm
errorNorm = lambda sol, ref : solNorm(sol - ref) / solNorm(ref)

In [ ]:
# Compute Errors 
y_ref = np.load(ERKRefSolFile)
y_fine = np.load(ERKFineSolFile)

errors_parareal = np.zeros((ks.size, Ngs.size))
for j in range(0, Ngs.size):
    for i in range(0, ks.size):
        y_par = np.load(ParSolFile(Ngs[j],ks[i], pTot, pFin))
        errors_parareal[i,j] = errorNorm(y_par, y_ref)

error_fine = errorNorm(y_fine, y_ref)

In [ ]:
# Compute Times
time_ref = extractTotalTime(ERKFineTimingFile)
time_fine = extractTotalTime(ERKFineTimingFile)

times_parareal = np.zeros((ks.size, Ngs.size))
for j in range(0, Ngs.size):
    for i in range(0, ks.size):
        times_parareal[i,j] = extractTotalTime(ParTimingFile(Ngs[j],ks[i], pTot, pFin))
    
speedup = time_fine / times_parareal;
savemat(f"vp-speedup", {"speedup" : speedup});

In [ ]:
plt.semilogy(ks, errors_parareal)
plt.semilogy(ks[[0, -1]], error_fine * np.array([1, 1]), 'k--')
plt.title('Error vs Iteration')
plt.show()

In [ ]:
plt.plot(speedup) 
plt.title('Speedup')
plt.show()

In [ ]:
plt.plot(ks, times_parareal)
plt.plot(ks, time_fine * np.ones(ks.shape), 'k--')
plt.title("Time vs Iteration")
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.linspace(0, 8*np.pi,y_ref.shape[1])
Y = np.linspace(-8*np.pi, 8*np.pi,y_ref.shape[0])
X, Y = np.meshgrid(X, Y)

surf = ax.plot_surface(X,Y, sol2Realspace(y_ref), cmap=cm.coolwarm,linewidth=0,rstride=1,cstride=1)#,rstride=10,cstride=10)
ax.view_init(elev=90., azim=0)